In [1]:
from glob import glob

In [12]:
stripfiles = sorted(glob('meshes/*strip*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]
print('    ', len(stripfiles))
print(stripfiles[0])
print(stripfiles[-1])

     2
meshes\frame96000_strip_s20_666.obj
meshes\frame96000_strip_s20_766.obj


In [13]:
def indadd(istr, amount):
    parts = [str(int(val) + amount) for val in istr.split()[1:]]
    return 'f ' + ' '.join(parts)

print(indadd('f 1 2 3 ', 10))

def norm(ww, vert):
    coords = [float(val) / float(ww.split('# w ')[1]) for val in vert.split()[1:]]
    return 'v ' + ' '.join([str(val) for val in coords])

print(norm(*('# w 2', 'v 2 2 2')))

f 11 12 13
v 1.0 1.0 1.0


In [14]:
verts = []
inds = []
verts_normed = []
verts_og = []

voffset = 0
FROM =0
UPTO = 150
for fname in stripfiles[FROM:UPTO]:
    with open(fname) as fl:
        raw = fl.read()
    lines = raw.split('\n')
    lines = [ll for ll in lines if ll]
    vs = [ll for ll in lines if ll[0] == 'v']
    
    ws = [ll for ll in lines if ll[:3] == '# w']
    os = [ll.replace('# o', 'v') for ll in lines if ll[:3] == '# o']
    fs = [ll for ll in lines if ll[0] == 'f']
    print('%s: %d verts, %d faces, %d ws' % (fname, len(vs), len(fs), len(ws)))
    
    verts_normed += [norm(*pair) for pair in zip(ws, vs)]
    verts_og += os
    verts += ['# ' + fname]
    verts += vs
    inds += [indadd(fdef, voffset) for fdef in fs]
    voffset += len(vs)

print('Total:  %d verts,  %d faces' % (len(verts), len(inds)))

meshes\frame96000_strip_s20_666.obj: 268 verts, 664 faces, 268 ws
meshes\frame96000_strip_s20_766.obj: 253 verts, 764 faces, 253 ws
Total:  523 verts,  1428 faces


In [15]:
with open('dump.obj', 'w') as fl:
    fl.write('\n'.join(verts))
    fl.write('\n')
    fl.write('\n'.join(inds))

In [16]:
with open('dump_og.obj', 'w') as fl:
    fl.write('\n'.join(verts_og))
    fl.write('\n')
    fl.write('\n'.join(inds))